In [1]:
import torch
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import os 
import time
from nilearn import plotting
from brainiak.searchlight.searchlight import Searchlight
from brainiak.fcma.preprocessing import prepare_searchlight_mvpa_data
from brainiak import io
from pathlib import Path
from shutil import copyfile
from scipy.stats import pearsonr
%matplotlib inline

### get paths to fmri data, must be 4-d

In [2]:
DATA_DIR = "/media/yoel/second_ssd/neu502b/movie_nii"
FILES = [os.path.join(DATA_DIR, x) for x in os.listdir(DATA_DIR) if x.endswith(".nii")]
FILES

['/media/yoel/second_ssd/neu502b/movie_nii/sherlock_movie_s1.nii',
 '/media/yoel/second_ssd/neu502b/movie_nii/sherlock_movie_s10.nii',
 '/media/yoel/second_ssd/neu502b/movie_nii/sherlock_movie_s11.nii',
 '/media/yoel/second_ssd/neu502b/movie_nii/sherlock_movie_s12.nii',
 '/media/yoel/second_ssd/neu502b/movie_nii/sherlock_movie_s13.nii',
 '/media/yoel/second_ssd/neu502b/movie_nii/sherlock_movie_s14.nii',
 '/media/yoel/second_ssd/neu502b/movie_nii/sherlock_movie_s15.nii',
 '/media/yoel/second_ssd/neu502b/movie_nii/sherlock_movie_s16.nii',
 '/media/yoel/second_ssd/neu502b/movie_nii/sherlock_movie_s17.nii',
 '/media/yoel/second_ssd/neu502b/movie_nii/sherlock_movie_s2.nii',
 '/media/yoel/second_ssd/neu502b/movie_nii/sherlock_movie_s3.nii',
 '/media/yoel/second_ssd/neu502b/movie_nii/sherlock_movie_s4.nii',
 '/media/yoel/second_ssd/neu502b/movie_nii/sherlock_movie_s5.nii',
 '/media/yoel/second_ssd/neu502b/movie_nii/sherlock_movie_s6.nii',
 '/media/yoel/second_ssd/neu502b/movie_nii/sherlock_mo

### get paths to the mask

In [3]:
MASK_PATH = "/media/yoel/second_ssd/neu502b/reqs/3mm_mask.nii.gz"

### load in data

In [4]:
mask_obj = nib.load(MASK_PATH)
bold_obj = nib.load(FILES[0])

### define other variables for the searchlight as shown in the brainiak tutorial

In [5]:
sl_rad = 1 # searchlight radius
max_blk_edge = 3 # not sure 
pool_size = 1 # not sure
mask = np.array(mask_obj.dataobj).astype(bool)
data = np.array(bold_obj.dataobj)

# truncate the data to part2
data = data[:, :, :, 946:]

# get activations correlation matrix
NNCOR_DIR = "/media/yoel/second_ssd/neu502b/results/nncormats"
NNCOR_FILES = [os.path.join(NNCOR_DIR, x) for x in os.listdir(NNCOR_DIR) if x.endswith(".npy")]
bcvar = np.load(NNCOR_FILES[-1])
print(bcvar)
print(NNCOR_FILES[-1])

# Create the searchlight object
sl = Searchlight(sl_rad=sl_rad, max_blk_edge=max_blk_edge)

[[1.         0.9715755  0.96752642 ... 0.15801158 0.14926232 0.15346959]
 [0.9715755  1.         0.97972615 ... 0.15466913 0.14005584 0.14180969]
 [0.96752642 0.97972615 1.         ... 0.17252008 0.15755962 0.15507047]
 ...
 [0.15801158 0.15466913 0.17252008 ... 1.         0.83300622 0.36757448]
 [0.14926232 0.14005584 0.15755962 ... 0.83300622 1.         0.39343924]
 [0.15346959 0.14180969 0.15507047 ... 0.36757448 0.39343924 1.        ]]
/media/yoel/second_ssd/neu502b/results/nncormats/p2relu9cormat.npy


In [6]:
sl.distribute([data], mask)
sl.broadcast(bcvar)

In [7]:
def rsa(data, mask, sl_rad, bcvar): 
    if np.sum(mask) < 2:
        return 0.0
    data4D = data[0]
    bolddata_sl = data4D[mask, :].T  
    human = np.corrcoef(bolddata_sl)
    alexnet = bcvar
    hvec = human[np.triu(np.ones(human.shape), k=10).astype(bool)]
    avec = alexnet[np.triu(np.ones(alexnet.shape), k=10).astype(bool)]
    nnans = np.isnan(hvec).sum()
    if nnans:
        print(nnans)
    return pearsonr(hvec, avec)[0]

In [8]:
print("Begin Searchlight\n")
sl_result = sl.run_searchlight(rsa, pool_size=pool_size)

Begin Searchlight



/media/yoel/second_ssd/torch/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]


520710


ValueError: array must not contain infs or NaNs